In [3]:
try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')

import time
import numpy as np

def normalizeAngle(angle):
    return np.mod(angle+np.pi, 2*np.pi) - np.pi

In [4]:
####################################################################################
#                                                                                  #
#                 LEMBRE-SE QUE A SIMULAÇÃO DEVE ESTAR EM EXECUÇÃO!                #
#                                                                                  #
####################################################################################

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID != -1:
    print ('Connected to remote API server')

    robotname = 'Pioneer_p3dx'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotname, sim.simx_opmode_oneshot_wait)     
        
    returnCode, l_wheel = sim.simxGetObjectHandle(clientID, robotname + '_leftMotor', sim.simx_opmode_oneshot_wait)
    returnCode, r_wheel = sim.simxGetObjectHandle(clientID, robotname + '_rightMotor', sim.simx_opmode_oneshot_wait)    
    
    # Handles para os sonares
    returnCode, sonar_front = sim.simxGetObjectHandle(clientID, robotname + '_ultrasonicSensor5', sim.simx_opmode_oneshot_wait)
    returnCode, sonar_right = sim.simxGetObjectHandle(clientID, robotname + '_ultrasonicSensor7', sim.simx_opmode_oneshot_wait)
       
    # Específico do robô   
    L = 0.331
    r = 0.09751
    
    following = False

    q = np.array([0,0])
    qf = np.array([3,6])
    qgoal = qf
    it = 0
    err = np.inf
    err_obs = np.inf
    while True:
        # Pos, Ori
        returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
        returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)  
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])

        dx, dy = qf - robotConfig[:2]
        err = np.sqrt(dx**2 + dy**2)

        # Fazendo leitura dos sensores
        returnCode, detected_front, point_front, *_ = sim.simxReadProximitySensor(clientID, sonar_front, sim.simx_opmode_oneshot_wait)
        returnCode, detected_right, point_right, *_ = sim.simxReadProximitySensor(clientID, sonar_right, sim.simx_opmode_oneshot_wait)
        
        # Velocidades iniciais
        v = .4
        w = 0

        obstacle_in_front = (detected_front and np.linalg.norm(point_front) < .5)
        obstacle_in_right = (detected_right and np.linalg.norm(point_right) < .5)
        
        # Controle
        if obstacle_in_front:
            if (it == 0):
                #goal = robotConfig[:2]
                qgoal = robotConfig[:2]
                it = it + 1
            v = 0
            w = np.deg2rad(30)
            following = True
        else: 
            if obstacle_in_right:
                w = np.deg2rad(10)
            elif following:
                v = .1
                w = np.deg2rad(-30)

        # Cond. Parada
        #if qgoal.all() == robotConfig.all() + 0.2 or qgoal.all() == robotConfig.all() - 0.2: break
        dxo, dyo = qgoal - robotConfig[:2]
        err_obs = np.sqrt(dxo**2 + dyo**2)
        print("qgoal: ", qgoal, " rc: ", robotConfig[:2], " it: ", it, "err_obs: ", err_obs)
        if (following):
            it = it + 1
            if (it > 50 and err_obs < 0.4):
                break

        # Cinemática Inversa
        wr = ((2.0 * v) + (w * L)) / (2.0 * r)
        wl = ((2.0 * v) - (w * L)) / (2.0 * r)    
        
        # Enviando velocidades
        sim.simxSetJointTargetVelocity(clientID, l_wheel, wl, sim.simx_opmode_oneshot_wait)
        sim.simxSetJointTargetVelocity(clientID, r_wheel, wr, sim.simx_opmode_oneshot_wait)
        
        # print(robotPos)
        # print(robotOri)
       
    sim.simxSetJointTargetVelocity(clientID, r_wheel, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, l_wheel, 0, sim.simx_opmode_oneshot_wait)
        
    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
    
else:
    print ('Failed connecting to remote API server')
    
print ('Program ended')

Program started
Connected to remote API server
qgoal:  [3 6]  rc:  [-2.96988058 -2.57527661]  it:  0 err_obs:  10.448676621646172
qgoal:  [3 6]  rc:  [-2.96731782 -2.57312703]  it:  0 err_obs:  10.445448245782353
qgoal:  [3 6]  rc:  [-2.89454007 -2.52592039]  it:  0 err_obs:  10.365178298970639
qgoal:  [3 6]  rc:  [-2.80300379 -2.46275663]  it:  0 err_obs:  10.261242751523884
qgoal:  [3 6]  rc:  [-2.7183013  -2.39429498]  it:  0 err_obs:  10.156926596099751
qgoal:  [3 6]  rc:  [-2.63035822 -2.31287146]  it:  0 err_obs:  10.04015764429643
qgoal:  [3 6]  rc:  [-2.54114938 -2.23267674]  it:  0 err_obs:  9.92377462491651
qgoal:  [3 6]  rc:  [-2.4519546  -2.15247583]  it:  0 err_obs:  9.807480370488014
qgoal:  [3 6]  rc:  [-2.36272025 -2.07226276]  it:  0 err_obs:  9.691243192804015
qgoal:  [3 6]  rc:  [-2.27350569 -1.99207306]  it:  0 err_obs:  9.575128929683023
qgoal:  [3 6]  rc:  [-2.18431163 -1.91187978]  it:  0 err_obs:  9.459118810958735
qgoal:  [3 6]  rc:  [-2.09515047 -1.8316828 ]  